In [8]:
import os
import sys

import pandas as pd
import numpy as np
from datetime import datetime as dt
import calendar
import ee
import matplotlib.pyplot as plt

ee.Initialize()

### To do
average also for annual with only months T>0C

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
data_dir = '/Users/maoyabassiouni/Documents/DATA/Projects/OLNI'

# Combine annual files

In [12]:
period_y = [1990, 2016]

In [13]:
df_locs = pd.read_csv(os.path.join(data_dir, 'sites', 'all_icp_sites.csv'))
locations = list(zip(df_locs.lon, df_locs.lat, df_locs.code_tree_species))

In [15]:
#scale
#https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD15A3H
data = []
for year in range(2002, 2016):
    modis_y = pd.read_csv(os.path.join(data_dir, 'grid_data_extract/', 'MCD15A3H061_extract_buffer', 'modis_sites_%s.csv' % year))
    #modis_y = modis_y[(modis_y['FparLai_QC']==0)]
    modis_y = modis_y.drop(['FparLai_QC', 'Unnamed: 0', 'doy'], axis=1)
    modis_y['timestamp'] = [dt.strptime(dti, '%Y-%m-%d %H:%M:%S')
                                 for dti in modis_y['timestamp'].values]
    modis_y = modis_y.groupby(['Lon', 'Lat', 'timestamp']).mean()
    modis_y['Fpar'] = modis_y['Fpar'] * 0.01
    modis_y['Lai'] = modis_y['Lai'] * 0.1
    
    data.append(modis_y)
modis_combined = pd.concat(data).reset_index()
modis_combined = modis_combined.groupby(['Lon', 'Lat', 'timestamp']).mean().dropna()
modis_combined.to_csv(os.path.join(data_dir, 'grid_data_extract/', 'modis_combined.csv'))

In [7]:
modis_combined.head(5)

year  month       Lai      Fpar
Lon       Lat    timestamp                                             
-8.971111 43.025 2002-07-07 17:00:00  2002.0    7.0  3.178154  0.719398
                 2002-07-15 17:00:00  2002.0    7.0  3.168989  0.725172
                 2002-07-19 17:00:00  2002.0    7.0  3.345624  0.758337
                 2002-07-23 17:00:00  2002.0    7.0  3.291089  0.753457
                 2002-07-27 17:00:00  2002.0    7.0  3.109572  0.737000

In [18]:
modis_vc = pd.read_csv(os.path.join(data_dir, 'grid_data_extract/', 'MOD44B_extract', 'modis_vc_sites.csv'))
modis_vc = modis_vc.drop(['Unnamed: 0', 'month', 'doy',  'year'], axis=1)

modis_vc = modis_vc[modis_vc['Quality']==0]
modis_vc = modis_vc.groupby(['Lon', 'Lat']).mean()
modis_vc['f_nontree'] = modis_vc['Percent_NonTree_Vegetation'] / 100
modis_vc['f_tree'] = modis_vc['Percent_Tree_Cover'] / 100
modis_vc['f_nonveg'] = modis_vc['Percent_NonVegetated'] / 100
modis_vc['tot'] = modis_vc['f_nontree'] + modis_vc['f_tree'] + modis_vc['f_nonveg']
modis_vc['f_veg']= 1 - modis_vc['f_nonveg']

modis_vc.to_csv(os.path.join(data_dir, 'grid_data_extract/',  'modis_vc_avg.csv'))

In [19]:
modis_vc.head(5)

,,Percent_Tree_Cover,Percent_NonTree_Vegetation,Percent_NonVegetated,Quality,f_nontree,f_tree,f_nonveg,tot,f_veg
Lon,Lat,,,,,,,,,
-8.971111,43.025000,49.181818,41.454545,9.363636,0.0,0.414545,0.491818,0.093636,1.0,0.906364
-8.962500,43.028889,36.090909,52.363636,11.545455,0.0,0.523636,0.360909,0.115455,1.0,0.884545
-8.961667,39.790833,37.789474,50.684211,11.526316,0.0,0.506842,0.377895,0.115263,1.0,0.884737
-8.825000,43.256167,35.909091,50.454545,13.636364,0.0,0.504545,0.359091,0.136364,1.0,0.863636
-8.772500,41.821333,34.789474,53.526316,11.684211,0.0,0.535263,0.347895,0.116842,1.0,0.883158


In [21]:
#units
#https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY
data = []
for year in range(period_y[0], period_y[1] + 1):
    era5_y = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'ERA5_extract', 'era5_sites_%s.csv' % year))
    era5_y = era5_y.drop(['Unnamed: 0', 'doy'], axis=1)
    era5_y['timestamp'] = [dt.strptime(dti, '%Y-%m-%d %H:%M:%S')
                                 for dti in era5_y['timestamp'].values]
    era5_y = era5_y.groupby(['Lon', 'Lat', 'timestamp']).mean()
    data.append(era5_y)
era5_combined = pd.concat(data).reset_index()
era5_combined['mean_2m_air_temperature'] = era5_combined['mean_2m_air_temperature'] - 273.15
era5_combined['total_precipitation'] = era5_combined['total_precipitation']        
era5_combined.loc[era5_combined['total_precipitation'] < 0.1/1000, 'total_precipitation'] = 0
era5_combined = era5_combined.groupby(['Lon', 'Lat', 'timestamp']).mean()
era5_combined.to_csv(os.path.join(data_dir, 'grid_data_extract', 'era5_combined.csv'))


In [22]:
era5_combined.head(3)

year  month  total_precipitation  \
Lon       Lat       timestamp                                                 
-9.553889 53.758333 1990-01-01 16:00:00  1990.0    1.0             0.000734   
                    1990-01-02 16:00:00  1990.0    1.0             0.003672   
                    1990-01-03 16:00:00  1990.0    1.0             0.000000   

                                         surface_pressure  \
Lon       Lat       timestamp                               
-9.553889 53.758333 1990-01-01 16:00:00      99983.554688   
                    1990-01-02 16:00:00      99766.375000   
                    1990-01-03 16:00:00     100266.843750   

                                         mean_2m_air_temperature  
Lon       Lat       timestamp                                     
-9.553889 53.758333 1990-01-01 16:00:00                 6.712366  
                    1990-01-02 16:00:00                 6.335931  
                    1990-01-03 16:00:00                 5.827264

In [56]:
def mean_DT_airT(tmmn, tmmx, lat, solar_declin):
    x = - np.tan(lat) * np.tan(solar_declin)
    t1 = (tmmx + 273.15) * (1 /2 + ((1 - x**2)**0.5) / (2 * np.arccos(x))) - 273.15
    t2 = (tmmn + 273.15) * (1 /2 - ((1 - x**2)**0.5) / (2 * np.arccos(x))) - 273.15
    return t1 + t2

In [24]:
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
# scale and by month conversions
data = []
for year in range(period_y[0], period_y[1] + 1):
    tc_y = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'TC_extract', 'TC_sites_%s.csv' % (year)))
    tc_y = tc_y.drop(['Unnamed: 0', 'doy'], axis=1)
    tc_y['timestamp'] = [dt.strptime(dti, '%Y-%m-%d %H:%M:%S')
                                 for dti in tc_y['timestamp'].values]
    tc_y['month'] = [dti.astype('datetime64[M]').astype(int) % 12 + 1 for dti in tc_y['timestamp'].values]
    tc_y = tc_y.groupby(['Lon', 'Lat', 'timestamp']).mean()
    data.append(tc_y)
tc_combined = pd.concat(data).reset_index()
tc_combined['days_month'] = [calendar.monthrange(np.int(y), np.int(m))[1] for y, m in zip(tc_combined['year'], tc_combined['month'])]
for band, scale in [['srad', 0.1], ['pet', 0.1],
                    ['aet', 0.1], ['def', 0.1],
                    ['pdsi', 0.01], ['pr', 1],  
                    ['ro', 1], ['soil', 0.1], 
                    ['tmmn', 0.1], ['tmmx', 0.1], 
                    ['vap', 0.001], ['vpd', 0.01], ['vs', 0.01]]:
    tc_combined[band] = tc_combined[band] * scale
for k in ['aet', 'def',  'pr', 'pet', 'ro', 'soil']:
    tc_combined[k] = tc_combined[k] / 1000 / tc_combined['days_month']
tc_combined.loc[tc_combined['pr'] < 0.1/1000, 'pr'] = 0
for k in ['vap', 'vpd']:
    tc_combined[k] = tc_combined[k] * 1000

tc_combined['tmean'] = (tc_combined['tmmx'] + tc_combined['tmmn']) / 2 
    
tc_combined = tc_combined.groupby(['Lon', 'Lat', 'timestamp']).mean()

tc_combined.to_csv(os.path.join(data_dir, 'grid_data_extract',  'tc_combined.csv'))

In [25]:
tc_combined.head(5)

year  month   srad       pet  \
Lon       Lat       timestamp                                             
-9.553889 53.758333 1990-01-31 16:00:00  1990.0    1.0   26.5  0.000745   
                    1990-02-28 16:00:00  1990.0    2.0   44.1  0.001232   
                    1990-03-31 16:00:00  1990.0    3.0   82.3  0.001519   
                    1990-04-30 17:00:00  1990.0    4.0  166.3  0.002077   
                    1990-05-31 17:00:00  1990.0    5.0  218.0  0.002965   

                                              aet       def  pdsi        pr  \
Lon       Lat       timestamp                                                 
-9.553889 53.758333 1990-01-31 16:00:00  0.000745  0.000000 -0.83  0.006194   
                    1990-02-28 16:00:00  0.001218  0.000000  3.02  0.007750   
                    1990-03-31 16:00:00  0.001519  0.000000  2.19  0.002387   
                    1990-04-30 17:00:00  0.002077  0.000000  2.26  0.002667   
                    1990-05-31 17:00:00  0.001900  0.001061  1.21  0.001032   

                                               ro      soil  tmmn  tmmx  \
Lon       Lat       timestamp                                             
-9.553889 53.758333 1990-01-31 16:00:00  0.005452  0.001445   4.0   9.5   
                    1990-02-28 16:00:00  0.006500  0.001600   3.4   9.5   
                    1990-03-31 16:00:00  0.000871  0.001445   5.9  11.1   
                    1990-04-30 17:00:00  0.000567  0.001493   4.2  12.0   
                    1990-05-31 17:00:00  0.000065  0.000532   8.3  16.7   

                                            vap    vpd     vs  days_month  \
Lon       Lat       timestamp                                               
-9.553889 53.758333 1990-01-31 16:00:00   855.0  150.0   8.59        31.0   
                    1990-02-28 16:00:00   777.0  210.0  10.13        28.0   
                    1990-03-31 16:00:00   893.0  240.0   8.49        31.0   
                    1990-04-30 17:00:00   841.0  280.0   6.34        30.0   
                    1990-05-31 17:00:00  1110.0  390.0   3.95        31.0   

                                         tmean  
Lon       Lat       timestamp                   
-9.553889 53.758333 1990-01-31 16:00:00   6.75  
                    1990-02-28 16:00:00   6.45  
                    1990-03-31 16:00:00   8.50  
                    1990-04-30 17:00:00   8.10  
                    1990-05-31 17:00:00  12.50

# Determine growing season months & peak

In [39]:
modis_combined = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'modis_combined.csv'))

gs_percentile = 100
gs_frac = 0.5
locs_gs_sel = []
for lon, lat, sp_id in locations:
    df_lai_0 = modis_combined[(modis_combined['Lon']==lon) & (modis_combined['Lat']==lat)]
    #plt.plot(df_lai_0['doy'], df_lai_0['Lai'])
    #plt.show()
    #print(lon, lat , len(df_lai_0[['Lai']].dropna()))
    #lai_len.append(len(df_lai_0[['Lai']].dropna()))
    if len(df_lai_0[['Lai']].dropna())>100:
        lai = df_lai_0['Lai'].values

        date = [dt.strptime(dti, '%Y-%m-%d %H:%M:%S')
                                 for dti in df_lai_0['timestamp'].values]
        
        df_lai = pd.DataFrame({'LAI': lai}, index=date)
        df_lai['smooth_LAI'] = np.nan
        df_lai = df_lai.resample('D').mean()
        df_lai = df_lai.interpolate()
        df_lai = df_lai.reindex(pd.date_range(start='2002-01-01',
                                            end='2021-12-31',
                                            freq='D'))

        df_lai = df_lai[~((df_lai.index.month == 2) & (df_lai.index.day == 29))]
        df_lai['Year'] = df_lai.index.year
        years = [y for y in list(set( df_lai['Year'].values))]
        doy = []
        for y in years:
            doy = np.concatenate((doy, range(1,366)), axis=None)
        doy = [np.int(di) for di in doy]
        df_lai['DOY'] = doy

        m_avg_lai = []
        for c, year in enumerate(years):
            df_lai_ii = df_lai[df_lai['Year']==year]
            m_avg_lai.append(df_lai_ii['LAI'].values)
        m_avg_lai = zip(*m_avg_lai)
        m_avg_lai = [np.nanmean(mi) for mi in m_avg_lai]
        df_lai['smooth_LAI'] = [m_avg_lai[di-1] for di in df_lai['DOY'].values]
        m_avg_lai_r = np.convolve(df_lai['smooth_LAI'].values, np.ones(61) / 61, mode='same')
        m_avg_lai_r = list(m_avg_lai_r[365:365*2])
        m_avg_lai_r.insert(31 + 28, m_avg_lai_r[31+27])
        lai_dates = pd.date_range(start='2000-01-01',
                                  end='2000-12-31',
                                  freq='1D')
        df_lai['LAI_'] =  np.nan
        for month, day, lai in zip(lai_dates.month, lai_dates.day, m_avg_lai_r):
            df_lai.loc[(df_lai.index.day == day) & (df_lai.index.month == month), 'LAI_'] = lai

        df_lai['M'] = df_lai.index.month

        data_m = df_lai[['LAI_', 'M']].groupby('M').mean()
        #if np.nanmean(data_m['LAI_'])>0.5:
        #plt.plot(m_avg_lai_r)
        #plt.show()

        data_m['LAI_n'] = data_m['LAI_'] - np.min(data_m['LAI_'])
        th_m =  np.percentile(data_m['LAI_n'].dropna(), gs_percentile) * gs_frac
        data_m = data_m[(data_m['LAI_n'] >= th_m)]
        months_gs = list(data_m.index)
        gs_lai = np.mean(data_m['LAI_'])
        peak_lai = np.max(data_m['LAI_'])
        peak_month = data_m[data_m['LAI_']==np.max(data_m['LAI_'])].index[0]
        print(months_gs, gs_lai, peak_month, peak_lai)
        locs_gs_sel.append([lat, lon, sp_id, months_gs, gs_lai, peak_month, peak_lai])

[4, 5, 6, 7, 8, 9] 1.2797430968197554 6 1.4133934228882048
[4, 5, 6, 7, 8, 9] 1.2797430968197554 6 1.4133934228882048
[4, 5, 6, 7, 8, 9] 1.5283476667752902 6 1.7125321049936015
[3, 4, 5, 6, 7, 8] 0.903121280731749 4 0.9687723855146557
[4, 5, 6, 7, 8, 9] 1.4045343662864742 6 1.5227823248374444
[4, 5, 6, 7, 8, 9] 4.077978978770653 7 4.522980386797191
[4, 5, 6, 7, 8] 1.0829022462266304 6 1.222946184024547
[3, 4, 5, 6] 1.0200270083966598 5 1.0850514151387551
[2, 3, 4, 5] 1.049869367601654 4 1.1537337605542772
[2, 3, 4, 5] 1.0686679732782702 4 1.1763036448744235
[2, 3, 4, 5] 1.90376275897179 4 2.2635875240401244
[2, 3, 4, 5] 1.90376275897179 4 2.2635875240401244
[4, 5, 6, 7, 8, 9] 1.4370431515588276 6 1.5754247805547068
[5, 6, 7, 8, 9] 3.7418478869712013 6 4.328478068373356
[4, 5, 6, 7] 1.1034254419068303 5 1.1925423185597421
[1, 2, 3, 4, 5] 1.4304434192762463 3 1.6226023502917883
[3, 4, 5, 6, 7, 8] 1.4023515203851524 5 1.4649520096398077
[3, 4, 5, 6, 7, 8] 1.4267810731536235 5 1.4989250581

[4, 5, 6, 7, 8, 9, 10] 3.0031362474244605 6 3.4214568763645428
[5, 6, 7, 8, 9] 3.624300514139443 7 3.977912513614601
[5, 6, 7, 8, 9] 2.991896157278755 6 3.4328656405721154
[5, 6, 7, 8, 9, 10] 3.34898039596021 7 3.8383461730173356
[5, 6, 7, 8, 9] 2.542915968967631 7 2.885939988660209
[5, 6, 7, 8, 9] 2.1442427417551153 7 2.5671292527107967
[5, 6, 7, 8, 9] 2.1442427417551153 7 2.5671292527107967
[5, 6, 7, 8, 9] 2.1442427417551153 7 2.5671292527107967
[5, 6, 7, 8, 9] 2.1442427417551153 7 2.5671292527107967
[5, 6, 7, 8, 9] 4.187996704911081 7 4.76175391609447
[5, 6, 7, 8, 9] 2.3063190245921468 7 2.7595772559192664
[5, 6, 7, 8, 9] 3.6042363487281293 7 4.239391322768374
[5, 6, 7, 8, 9] 3.6042363487281293 7 4.239391322768374
[5, 6, 7, 8, 9] 3.6573782446813268 6 4.299754143707261
[5, 6, 7, 8, 9] 3.429355270780291 7 3.9801306603272018
[5, 6, 7, 8, 9] 1.373537798540037 7 1.7122253434461563
[5, 6, 7, 8, 9, 10] 3.0394962813514845 6 3.456695310844791
[5, 6, 7, 8, 9, 10] 3.615174497138678 7 4.2945856

[5, 6, 7, 8, 9] 3.0185447358686863 7 3.4358162329143784
[5, 6, 7, 8, 9] 3.3925376155690317 6 3.9091009444440434
[5, 6, 7, 8, 9] 2.849711013864586 6 3.2135184604315317
[5, 6, 7, 8, 9] 2.849711013864586 6 3.2135184604315317
[5, 6, 7, 8, 9] 2.849711013864586 6 3.2135184604315317
[4, 5, 6, 7, 8, 9, 10] 2.857662385103516 8 3.2888956127249163
[5, 6, 7, 8, 9] 4.042179544950681 6 4.476766395303837
[5, 6, 7, 8, 9] 4.042179544950681 6 4.476766395303837
[5, 6, 7, 8, 9] 3.629520060017891 6 4.226568766660477
[5, 6, 7, 8, 9] 2.586173240157186 6 2.904124572354031
[4, 5, 6, 7, 8, 9, 10] 2.54130877394225 6 3.019172125495155
[5, 6, 7, 8] 1.889866462297154 6 2.3183450538356953
[5, 6, 7, 8, 9, 10] 3.503079526324288 6 3.8938123448927717
[5, 6, 7, 8, 9] 1.8795705085780186 6 2.295345094875564
[5, 6, 7, 8, 9] 4.412380119141811 8 4.740364432968934
[5, 6, 7, 8] 2.3648518273734247 6 2.8248262713906924
[5, 6, 7, 8, 9] 2.448610060473393 6 2.909355898160881
[5, 6, 7, 8] 1.8350289805273894 6 2.254834663885774
[5, 6,

[5, 6, 7, 8, 9] 2.145644513646285 6 2.4917482841515906
[5, 6, 7, 8, 9] 3.695531764512514 6 4.190334606360069
[5, 6, 7, 8] 2.0532876268171014 6 2.436429162134625
[5, 6, 7, 8] 2.505465873963202 6 2.9611292769603854
[5, 6, 7] 2.444664685661703 6 2.8513845505587243
[5, 6, 7, 8, 9] 2.6767926395117705 6 3.130250896605398
[5, 6, 7, 8, 9] 3.7613019418251596 6 4.202428052376944
[5, 6, 7, 8, 9] 2.484827519813682 7 2.766389046631659
[5, 6, 7, 8, 9] 3.6080632573603326 6 4.015419251133328
[4, 5, 6, 7, 8, 9] 1.8900832968545824 6 2.193916122406894
[5, 6, 7, 8, 9] 3.598284527979394 6 3.9662986429358336
[5, 6, 7, 8, 9] 3.0875625039667116 6 3.4409354155958507
[5, 6, 7, 8] 2.330812220035682 6 2.9186595851974952
[5, 6, 7, 8, 9] 2.5746159613499806 6 2.8876248801126994
[5, 6, 7, 8, 9] 2.0285436014600227 6 2.373608255548597
[5, 6, 7, 8, 9] 2.4967437747541363 6 3.2697712603815305
[5, 6, 7, 8, 9] 2.6824268465669983 7 3.0194289829734626
[5, 6, 7, 8, 9] 2.8727701507720718 6 3.345924128345381
[5, 6, 7, 8, 9] 2.47

[5, 6, 7, 8] 1.8882295063162104 6 2.300894141782481
[5, 6, 7, 8, 9] 3.106309586977912 6 3.4873898889058794
[5, 6, 7, 8, 9] 3.106309586977912 6 3.4873898889058794
[5, 6, 7, 8, 9] 2.1711493814553866 7 2.343505301682918
[5, 6, 7, 8, 9] 2.5622952353166397 6 2.9639105008275575
[5, 6, 7, 8, 9] 2.444392548086261 7 2.651876801493989
[5, 6, 7, 8, 9] 2.444392548086261 7 2.651876801493989
[5, 6, 7, 8, 9, 10] 2.035198100669907 7 2.2463837335812222
[5, 6, 7, 8, 9] 2.8610505226986747 7 3.2678419239331165
[5, 6, 7, 8, 9] 2.8016880650152673 6 2.9652915570325677
[5, 6, 7, 8, 9] 2.8016880650152673 6 2.9652915570325677
[5, 6, 7, 8, 9] 3.1944940274649034 6 3.5960815726102306
[5, 6, 7, 8, 9] 3.1944940274649034 6 3.5960815726102306
[5, 6, 7, 8, 9] 3.068399831696457 7 3.4768175463523807
[5, 6, 7, 8, 9] 3.210416663177489 7 3.6008409807385657
[5, 6, 7, 8, 9] 2.0278476147582047 7 2.3730993189430123
[5, 6, 7, 8, 9] 1.6645315603027289 7 1.9051818400292866
[5, 6, 7, 8, 9, 10] 2.6912650207581383 8 3.051942697410013

[5, 6, 7, 8, 9] 2.633055482010237 7 3.0366958954218064
[5, 6, 7, 8, 9] 3.0799833484884553 6 3.621187919196231
[5, 6, 7] 2.4053271194163233 6 2.8304416423058845
[5, 6, 7] 2.4053271194163233 6 2.8304416423058845
[5, 6, 7, 8] 2.946061460747609 6 3.6175040876642166
[5, 6, 7, 8, 9] 2.354263360704143 6 2.9054218586722262
[5, 6, 7, 8, 9] 2.5659189974328456 7 2.861774954474254
[5, 6, 7, 8, 9] 2.5367409485189514 7 2.8399546887299643
[4, 5, 6, 7, 8, 9, 10] 2.419527828393321 8 2.757979132455109
[5, 6, 7, 8, 9] 2.5036887508174237 7 2.932588497422825
[5, 6, 7, 8, 9] 2.5036887508174237 7 2.932588497422825
[5, 6, 7, 8, 9] 3.314006457535099 7 3.889648679312983
[5, 6, 7, 8, 9] 2.709784478091076 6 3.257461354015497
[6, 7, 8, 9] 2.1355171101004133 7 2.3267961445900562
[5, 6, 7, 8, 9] 2.4678544009065675 6 3.0048494704922137
[5, 6, 7, 8, 9] 2.4678544009065675 6 3.0048494704922137
[5, 6, 7, 8, 9] 2.4678544009065675 6 3.0048494704922137
[5, 6, 7, 8, 9] 2.4678544009065675 6 3.0048494704922137
[5, 6, 7, 8, 9] 

[6, 7, 8, 9] 2.4832252135471515 7 2.7821412152493723
[6, 7, 8, 9] 2.4832252135471515 7 2.7821412152493723
[6, 7, 8] 3.0673875498859204 7 3.390538232452031
[6, 7, 8] 2.8503414252475494 7 3.137597826820922
[6, 7, 8] 2.9069703718197797 7 3.2201447645870394
[6, 7, 8] 3.2255335287888247 7 3.6583036132248354
[6, 7, 8, 9] 1.5893749100541017 7 1.8785405344610813
[6, 7, 8, 9] 1.6079027529311487 7 1.8568395355736302
[6, 7, 8] 2.741103226093512 7 3.07447155839895
[6, 7, 8] 3.3669445203230377 7 3.70306454079799
[6, 7, 8, 9] 1.0924445090434824 7 1.3291808836074341
[6, 7, 8] 2.62228433644712 7 2.9735170376444144
[5, 6, 7, 8, 9, 10] 1.206564731866466 7 1.422745732657586
[5, 6, 7, 8, 9] 1.7725841655391772 7 2.1504411706683006
[5, 6, 7, 8, 9] 2.535724208820691 7 3.09775181514724
[6, 7, 8, 9] 1.6405436048360351 7 2.020652764214714
[6, 7, 8] 2.7265773621962395 7 3.004759730208446
[5, 6, 7, 8, 9] 1.4831197754908287 7 1.7951814902435441
[5, 6, 7, 8, 9] 2.084449695745505 7 2.565819519972951
[5, 6, 7, 8, 9] 

In [40]:
len(locs_gs_sel)

994

In [41]:
lat, lon, sp_id, months_gs, gs_lai, peak_gs_month, peak_lai = zip(*locs_gs_sel)
gs_df = pd.DataFrame({'Lat': lat, 'Lon': lon, 'sp_id': sp_id, 'months_gs': months_gs, 
                      'gs_lai': gs_lai, 'peak_gs_month': peak_gs_month, 'peak_lai': peak_lai})
gs_df.to_csv(os.path.join(data_dir, 'grid_data_extract', 'growing_season_locs_b.csv'))

# Process growing season annual timeseries

In [43]:
gs_df = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'growing_season_locs_b.csv'))

In [44]:
def stochastic_rf_char(rf):
    mu = np.nanmean(rf)
    var = np.nanvar(rf)
    l = 2 * mu ** 2 / var
    a = mu / l
    return a, l

In [45]:
period_y

[1990, 2016]

In [46]:
gs_df

,Unnamed: 0,Lat,Lon,sp_id,months_gs,gs_lai,peak_gs_month,peak_lai
0,0,34.929444,32.931111,121.0,"[4, 5, 6, 7, 8, 9]",1.279743,6,1.413393
1,1,34.929444,32.931111,129.0,"[4, 5, 6, 7, 8, 9]",1.279743,6,1.413393
2,2,34.952222,32.830556,129.0,"[4, 5, 6, 7, 8, 9]",1.528348,6,1.712532
3,3,35.013889,33.050000,121.0,"[3, 4, 5, 6, 7, 8]",0.903121,4,0.968772
4,4,35.020000,32.627500,121.0,"[4, 5, 6, 7, 8, 9]",1.404534,6,1.522782
...,...,...,...,...,...,...,...,...
989,989,67.999444,24.236111,118.0,"[6, 7, 8, 9]",1.146759,7,1.423312
990,990,68.995278,19.427500,118.0,"[6, 7, 8]",2.025214,7,2.366606
991,991,69.454167,30.036111,134.0,"[6, 7, 8, 9]",1.448153,7,1.832165
992,992,69.583056,28.898056,134.0,"[6, 7, 8, 9]",0.697611,7,0.841111


In [47]:
keys = ['mean_2m_air_temperature', 'surface_pressure', 'total_precipitation']
data_combined = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'era5_combined.csv'))
data_ym = data_combined.groupby(['Lon', 'Lat', 'year']).mean()
data_ym = data_ym.drop(['month',], axis=1)
rf_char_a = []
rf_char_gs = []
v_gs = []
v_gs_p = []
for lon, lat, year in data_ym.index:
    rf = data_combined[(data_combined['Lon']==lon) 
                       & (data_combined['Lat']==lat) 
                       & (data_combined['year']==year)]['total_precipitation'].values
    rf_char_a.append(stochastic_rf_char(rf))
    
    gsi = gs_df[(gs_df['Lon']==lon) & (gs_df['Lat']==lat)]
    if len(gsi.index)>=1:
        months_gs = gsi['months_gs'].values[0]
        months_gs = [np.int(m) for m in months_gs[1:-1].split(',')]
        peak_gs_month = gsi['peak_gs_month'].values[0]
        data_gs = data_combined[(data_combined['Lon']==lon) 
                       & (data_combined['Lat']==lat) 
                       & (data_combined['year']==year)]
        for m in range(1, 13):
            if m not in months_gs:
                data_gs = data_gs[data_gs['month']!=m]
        rf_c = stochastic_rf_char(data_gs['total_precipitation'].values)
        if (np.isnan(rf_c[0])) and (year<2017):
            print('rf_c nan', lon, lat, year)
        rf_char_gs.append(rf_c)
        v_gs.append([np.mean(data_gs[kk]) for kk in keys])
        data_gs_peak = data_gs[data_gs['month']==peak_gs_month]
        v_gs_p.append([np.mean(data_gs_peak[kk]) for kk in keys])
    else:
        print(len(gsi.index)==0, lon, lat, year)
        rf_char_gs.append([np.nan, np.nan])
        v_gs.append([np.nan for kk in keys])
        v_gs_p.append([np.nan for kk in keys])
rf_alpha, rf_lambda = zip(*rf_char_a)
rf_alpha_gs, rf_lambda_gs = zip(*rf_char_gs)
data_ym['rf_alpha'] = rf_alpha
data_ym['rf_lambda'] = rf_lambda
data_ym['rf_alpha_gs'] = rf_alpha_gs
data_ym['rf_lambda_gs'] = rf_lambda_gs
for v, kk in zip(zip(*v_gs), keys):
    data_ym['%s_gs' %kk] = v
for v, kk in zip(zip(*v_gs_p), keys):
    data_ym['%s_gs_peak' %kk] = v
data_ym = data_ym.reset_index()
data_ym = data_ym[data_ym['year']<=2016]
data_ym.to_csv(os.path.join(data_dir, 'grid_data_extract', 'era5_annual.csv'))

len(gsi.index)==0 13.4550922216667 58.7742688833333 1990.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1991.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1992.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1993.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1994.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1995.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1996.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1997.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1998.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 1999.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2000.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2001.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2002.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2003.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2004.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2005.0
len(gsi.index)==0 13.4550922216667 58.7742688833333 2006

In [48]:
data_ym

,Lon,Lat,year,total_precipitation,surface_pressure,mean_2m_air_temperature,rf_alpha,rf_lambda,rf_alpha_gs,rf_lambda_gs,mean_2m_air_temperature_gs,surface_pressure_gs,total_precipitation_gs,mean_2m_air_temperature_gs_peak,surface_pressure_gs_peak,total_precipitation_gs_peak
0,-9.553889,53.758333,1990.0,0.003526,100657.420222,10.275413,0.003170,1.112389,0.001943,1.120867,13.579172,101076.662224,0.002177,15.237696,101033.202873,0.002680
1,-9.553889,53.758333,1991.0,0.003167,100820.956122,9.968522,0.003369,0.940042,0.002309,1.001157,13.636737,101026.629340,0.002312,15.385089,100641.290827,0.003495
2,-9.553889,53.758333,1992.0,0.003799,100771.115779,9.833021,0.003195,1.189117,0.003586,1.062900,13.050939,100751.264910,0.003811,14.622592,100794.048135,0.004337
3,-9.553889,53.758333,1993.0,0.003366,100780.629409,9.626899,0.002678,1.256846,0.002279,1.270195,12.507955,100901.341197,0.002895,13.534856,100918.614163,0.002854
4,-9.553889,53.758333,1994.0,0.003683,100411.304281,9.836779,0.002313,1.592551,0.002310,1.219958,12.583319,100800.365605,0.002818,14.425981,100898.728327,0.002258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25278,36.435333,62.284333,2012.0,0.002289,99872.737001,1.887718,0.003140,0.729047,0.004393,0.854663,14.944338,99666.040166,0.003754,16.829439,99697.606351,0.003657
25279,36.435333,62.284333,2013.0,0.002134,99721.537457,3.831007,0.002899,0.736084,0.004276,0.706246,16.598631,99947.712806,0.003020,16.936197,99748.736139,0.002591
25280,36.435333,62.284333,2014.0,0.001915,100001.445655,3.684465,0.003205,0.597425,0.004906,0.614680,15.647411,99852.554772,0.003016,17.570230,100319.270917,0.002162
25281,36.435333,62.284333,2015.0,0.002223,99601.796254,4.127991,0.001927,1.153476,0.002285,1.270973,14.141446,99671.098251,0.002904,14.196073,99260.004032,0.003063


In [49]:
data_ym.dropna()

,Lon,Lat,year,total_precipitation,surface_pressure,mean_2m_air_temperature,rf_alpha,rf_lambda,rf_alpha_gs,rf_lambda_gs,mean_2m_air_temperature_gs,surface_pressure_gs,total_precipitation_gs,mean_2m_air_temperature_gs_peak,surface_pressure_gs_peak,total_precipitation_gs_peak
0,-9.553889,53.758333,1990.0,0.003526,100657.420222,10.275413,0.003170,1.112389,0.001943,1.120867,13.579172,101076.662224,0.002177,15.237696,101033.202873,0.002680
1,-9.553889,53.758333,1991.0,0.003167,100820.956122,9.968522,0.003369,0.940042,0.002309,1.001157,13.636737,101026.629340,0.002312,15.385089,100641.290827,0.003495
2,-9.553889,53.758333,1992.0,0.003799,100771.115779,9.833021,0.003195,1.189117,0.003586,1.062900,13.050939,100751.264910,0.003811,14.622592,100794.048135,0.004337
3,-9.553889,53.758333,1993.0,0.003366,100780.629409,9.626899,0.002678,1.256846,0.002279,1.270195,12.507955,100901.341197,0.002895,13.534856,100918.614163,0.002854
4,-9.553889,53.758333,1994.0,0.003683,100411.304281,9.836779,0.002313,1.592551,0.002310,1.219958,12.583319,100800.365605,0.002818,14.425981,100898.728327,0.002258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25278,36.435333,62.284333,2012.0,0.002289,99872.737001,1.887718,0.003140,0.729047,0.004393,0.854663,14.944338,99666.040166,0.003754,16.829439,99697.606351,0.003657
25279,36.435333,62.284333,2013.0,0.002134,99721.537457,3.831007,0.002899,0.736084,0.004276,0.706246,16.598631,99947.712806,0.003020,16.936197,99748.736139,0.002591
25280,36.435333,62.284333,2014.0,0.001915,100001.445655,3.684465,0.003205,0.597425,0.004906,0.614680,15.647411,99852.554772,0.003016,17.570230,100319.270917,0.002162
25281,36.435333,62.284333,2015.0,0.002223,99601.796254,4.127991,0.001927,1.153476,0.002285,1.270973,14.141446,99671.098251,0.002904,14.196073,99260.004032,0.003063


27

In [50]:
keys = ['Fpar', 'Lai']

data_combined = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'modis_combined.csv'))
data_ym = data_combined.groupby(['Lon', 'Lat', 'year']).mean()
data_ym = data_ym.drop(['month',], axis=1)
v_gs = []
v_gs_p = []
for lon, lat, year in data_ym.index:
    gsi = gs_df[(gs_df['Lon']==lon) & (gs_df['Lat']==lat)]
    if len(gsi.index)>=1:
        months_gs = gsi['months_gs'].values[0]
        months_gs = [np.int(m) for m in months_gs[1:-1].split(',')]
        peak_gs_month = gsi['peak_gs_month'].values[0]
        data_gs = data_combined[(data_combined['Lon']==lon) 
                       & (data_combined['Lat']==lat) 
                       & (data_combined['year']==year)]
        for m in range(1, 13):
            if m not in months_gs:
                data_gs = data_gs[data_gs['month']!=m]
        v_gs.append([np.mean(data_gs[kk]) for kk in keys])
        data_gs_peak = data_gs[data_gs['month']==peak_gs_month]
        v_gs_p.append([np.mean(data_gs_peak[kk]) for kk in keys])
    else:
        v_gs.append([np.nan for kk in keys])
        v_gs_p.append([np.nan for kk in keys])
for v, kk in zip(zip(*v_gs), keys):
    data_ym['%s_gs' %kk] = v
for v, kk in zip(zip(*v_gs_p), keys):
    data_ym['%s_gs_peak' %kk] = v
data_ym = data_ym.reset_index()
data_ym = data_ym[data_ym['year']<=2016]
data_ym.to_csv(os.path.join(data_dir, 'grid_data_extract', 'modis_annual.csv'))

In [51]:
data_ym.dropna()

,Lon,Lat,year,Lai,Fpar,Fpar_gs,Lai_gs,Fpar_gs_peak,Lai_gs_peak
0,-9.553889,53.758333,2002.0,2.237526,0.651952,0.700459,2.536991,0.676322,2.523129
1,-9.553889,53.758333,2003.0,2.004896,0.626840,0.672292,2.548369,0.606322,2.421144
2,-9.553889,53.758333,2004.0,1.844426,0.586257,0.644371,2.478900,0.622087,2.568462
3,-9.553889,53.758333,2005.0,1.995289,0.627917,0.683148,2.572854,0.797259,3.808345
4,-9.553889,53.758333,2006.0,1.872019,0.611906,0.657685,2.454790,0.783234,3.719422
...,...,...,...,...,...,...,...,...,...
12750,36.435333,62.284333,2011.0,1.551364,0.550848,0.729938,2.497747,0.804740,3.156673
12751,36.435333,62.284333,2012.0,1.487594,0.550705,0.713383,2.373106,0.721387,2.498036
12752,36.435333,62.284333,2013.0,1.525893,0.555219,0.726366,2.470474,0.737078,2.567109
12753,36.435333,62.284333,2014.0,1.588203,0.570116,0.717339,2.496644,0.796804,3.063400


In [52]:
keys = ['aet', 'def', 'month', 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'tmean', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs']
data_combined = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'tc_combined.csv'))
data_ym = data_combined.groupby(['Lon', 'Lat', 'year']).mean()
data_ym = data_ym.drop(['month',], axis=1)
v_gs = []
v_gs_p = []
for lon, lat, year in data_ym.index:
    gsi = gs_df[(gs_df['Lon']==lon) & (gs_df['Lat']==lat)]
    if len(gsi.index)>=1:
        months_gs = gsi['months_gs'].values[0]
        months_gs = [np.int(m) for m in months_gs[1:-1].split(',')]
        peak_gs_month = gsi['peak_gs_month'].values[0]
        data_gs = data_combined[(data_combined['Lon']==lon) 
                       & (data_combined['Lat']==lat) 
                       & (data_combined['year']==year)]
        for m in range(1, 13):
            if m not in months_gs:
                data_gs = data_gs[data_gs['month']!=m]
        v_gs.append([np.mean(data_gs[kk]) for kk in keys])
        data_gs_peak = data_gs[data_gs['month']==peak_gs_month]
        v_gs_p.append([np.mean(data_gs_peak[kk]) for kk in keys])
    else:
        v_gs.append([np.nan for kk in keys])
        v_gs_p.append([np.nan for kk in keys])
for v, kk in zip(zip(*v_gs), keys):
    data_ym['%s_gs' %kk] = v
for v, kk in zip(zip(*v_gs_p), keys):
    data_ym['%s_gs_peak' %kk] = v
data_ym = data_ym.reset_index()
data_ym = data_ym[data_ym['year']<=2016]
data_ym.to_csv(os.path.join(data_dir, 'grid_data_extract', 'tc_annual.csv'))

In [53]:
data_ym.dropna()

,Lon,Lat,year,srad,pet,aet,def,pdsi,pr,ro,...,pr_gs_peak,ro_gs_peak,soil_gs_peak,srad_gs_peak,tmean_gs_peak,tmmn_gs_peak,tmmx_gs_peak,vap_gs_peak,vpd_gs_peak,vs_gs_peak
0,-9.553889,53.758333,1990.0,116.645455,0.001818,0.001664,0.000153,0.665455,0.003667,0.001995,...,0.002194,0.000097,0.000510,200.3,15.50,11.6,19.4,1403.0,410.0,5.66
1,-9.553889,53.758333,1991.0,108.466667,0.001576,0.001413,0.000159,0.109167,0.003379,0.001963,...,0.002613,0.000129,0.000697,177.8,15.50,12.6,18.4,1465.0,330.0,5.74
2,-9.553889,53.758333,1992.0,104.741667,0.001585,0.001409,0.000176,0.300833,0.003577,0.002172,...,0.003452,0.000161,0.001023,172.4,14.85,11.6,18.1,1373.0,360.0,5.11
3,-9.553889,53.758333,1993.0,106.391667,0.001529,0.001513,0.000015,0.730000,0.002991,0.001474,...,0.002677,0.000161,0.001445,170.4,13.85,11.0,16.7,1312.0,300.0,6.09
4,-9.553889,53.758333,1994.0,103.950000,0.001554,0.001523,0.000028,2.589167,0.003834,0.002308,...,0.002677,0.000129,0.000952,174.7,14.55,11.3,17.8,1373.0,320.0,4.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25278,36.435333,62.284333,2012.0,96.416667,0.001165,0.001122,0.000010,1.705000,0.002193,0.001035,...,0.003226,0.000161,0.002458,201.6,16.65,11.6,21.7,1355.0,630.0,2.61
25279,36.435333,62.284333,2013.0,103.441667,0.001305,0.001149,0.000119,0.114167,0.001786,0.000660,...,0.002419,0.000129,0.000968,194.2,16.30,11.5,21.1,1285.0,650.0,2.34
25280,36.435333,62.284333,2014.0,104.075000,0.001287,0.001164,0.000096,-0.515833,0.001709,0.000591,...,0.001903,0.000097,0.001087,243.2,17.15,11.2,23.1,1269.0,810.0,2.04
25281,36.435333,62.284333,2015.0,96.933333,0.001204,0.001109,0.000062,-1.851667,0.001782,0.000622,...,0.002677,0.000129,0.001513,166.8,13.55,8.7,18.4,1097.0,530.0,2.24


In [55]:
#https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.txt
co2_df = [
          [1959,   315.98],
          [1960,   316.91],
          [1961,   317.64],
          [1962,   318.45],
          [1963,   318.99],
          [1964,   319.62],
          [1965,   320.04],
          [1966,   321.37],
          [1967,   322.18],
          [1968,   323.05],
          [1969,   324.62],
          [1970,   325.68],
          [1971,   326.32],
          [1972,   327.46],
          [1973,   329.68],
          [1974,   330.19],
          [1975,   331.12],
          [1976,   332.03],
          [1977,   333.84],
          [1978,   335.41],
          [1979,   336.84],
          [1980,   338.76],
          [1981,   340.12],
          [1982,   341.48],
          [1983,   343.15],
          [1984,   344.85],
          [1985,   346.35],
          [1986,   347.61],
          [1987,   349.31],
          [1988,   351.69],
          [1989,   353.20],
          [1990,   354.45],
          [1991,   355.70],
          [1992,   356.54],
          [1993,   357.21],
          [1994,   358.96],
          [1995,   360.97],
          [1996,   362.74],
          [1997,   363.88],
          [1998,   366.84],
          [1999,   368.54],
          [2000,   369.71],
          [2001,   371.32],
          [2002,   373.45],
          [2003,   375.98],
          [2004,   377.70],
          [2005,   379.98],
          [2006,   382.09],
          [2007,   384.02],
          [2008,   385.83],
          [2009,   387.64],
          [2010,   390.10],
          [2011,   391.85],
          [2012,   394.06],
          [2013,   396.74],
          [2014,   398.87],
          [2015,   401.01],
          [2016,   404.41],
          [2017,   406.76],
          [2018,   408.72],
          [2019,   411.66],
          [2020,   414.24],
          [2021,   416.45]]

ts, co2 = zip(*co2_df)
co2_df = pd.DataFrame({'year': ts, 'CO2': co2})
co2_df = co2_df[co2_df['year']<=2016]
co2_df.to_csv(os.path.join(data_dir, 'grid_data_extract', 'co2_annual.csv'))